In [1]:
import ruamel.yaml as ruyaml
import sys
import os

`ruamel.yaml` 的 `YAML()` 构造函数通过 `typ` 参数来控制其核心工作模式。这个模式决定了 YAML 实例在加载（load）和转储（dump）数据时的行为，尤其是在处理格式、注释和数据类型方面。

# typ 说明

## 1. `typ="rt"` (Round-Trip - 往返模式)

这是 `ruamel.yaml` 的默认模式，也是其最强大和最核心的功能。

- 它可以读取和写入核心目的：实现“往返”操作，即读取一个 `YAML` 文件，进行程序化修改，然后写回文件，同时最大限度地保留原始文件的所有细节。

- 主要特性：

    - 保留注释：行内和独立的注释都会被保留。

    - 保留格式：缩进、空行、块样式（block style）和流样式（flow style）等都会被保留。

    - 保留细节：包括引号的类型、锚点和别名等。

    - 安全性：此模式是完全安全的，不会执行任意代码。

    - 加载结果：`load() 方法返回的不是标准的 `dict` 和 `list`，而是 `ruamel.yaml` 自定义的 `CommentedMap` 和 `CommentedSeq` 对象。这些对象继承自 `dict` 和 `list`，但附加了存储注释和格式所需的信息。

- 适用场景：

    - 以编程方式修改配置文件（如 `config.yaml`、`CI/CD` 流水线文件）。

    - 例如，在 CI/CD 系统中，需要更新 Docker Compose (`docker-compose.yml`) 文件中的镜像版本。

    - 任何需要保持文件人类可读性和原始结构不变的场景。

In [2]:
yaml_str = """
# 服务器配置
server:
  host: 127.0.0.1  # 本机地址
  port: 8080
"""

In [3]:
yaml = ruyaml.YAML(typ="rt")  # 'rt' 是默认值，可以省略
yaml

In [4]:
data = yaml.load(yaml_str)
print(type(data))  # <class 'dict'>
print(data)

<class 'ruamel.yaml.comments.CommentedMap'>
{'server': {'host': '127.0.0.1', 'port': 8080}}


In [5]:
# 修改数据
data["server"]["port"] = 9000
data["server"].yaml_add_eol_comment("端口已被修改", "port")
data

{'server': {'host': '127.0.0.1', 'port': 9000}}

In [6]:
yaml.dump(data, sys.stdout)
# 可以看到，所有注释和结构都得到了完美保留，并且还能以编程方式添加新注释。

# 服务器配置
server:
  host: 127.0.0.1  # 本机地址
  port: 9000       # 端口已被修改


## 2. `typ="safe"` (Safe - 安全模式)

此模式是为了兼容 `PyYAML` 的 `safe_load/safe_dump` 而设计的。

- 核心目的：安全地加载 `YAML` 数据，将其转换为纯粹的、标准的 `Python` 对象。

- 主要特性：

    - 不保留注释和格式：加载和转储过程会丢失所有注释、空行和原始格式。

    - 安全性：与 `rt` 模式一样，此模式是安全的。

    - 加载结果：`load()` 方法返回标准的 `Python` 对象，即 `dict` 和 `list`。

- 适用场景：

    - 当你只关心 `YAML` 文件中的数据内容，而不需要写回或保留其原始格式时。

    - 作为数据源进行数据摄取和分析。

    - 需要将 `ruamel.yaml` 的行为与 `PyYAML` 的 `safe_load` 对齐时。

In [7]:
yaml_str = """
# 服务器配置
server:
  host: 127.0.0.1  # 本机地址
  port: 8080
"""

In [8]:
yaml = ruyaml.YAML(typ="safe")  # 明确指定安全模式
yaml

In [9]:
data = yaml.load(yaml_str)

print(type(data))  # <class 'dict'>
print(data)

<class 'dict'>
{'server': {'host': '127.0.0.1', 'port': 8080}}


In [10]:
# 转储时，所有注释和格式都丢失了
yaml.dump(data, sys.stdout)

server: {host: 127.0.0.1, port: 8080}


## 3. `typ="unsafe"` (Unsafe - 不安全模式)

此模式对应 `PyYAML` 中不安全的 `load()` 函数。强烈不推荐使用。

- 核心目的：加载可以执行任意 `Python` 代码的 `YAML` 数据。

- 主要特性：

    - 极不安全：它可以构造任何 `Python` 对象，并执行 `YAML` 文件中嵌入的任意代码。如果加载来源不可信的 `YAML` 文件，可能会导致严重的安全漏洞。

    - 加载结果：可以返回任何类型的 `Python` 对象。

- 适用场景：

    - 几乎没有。应极力避免。只应在完全信任 `YAML` 文件来源，并且确实需要序列化和反序列化复杂 `Python` 对象时才考虑，但通常有更安全的替代方案（如 `pickle` 用于内部数据，但同样有安全问题）。

示例（仅为演示，切勿在生产中使用）：

In [11]:
# 这个 YAML 字符串包含了执行 'whoami' 系统命令的指令
unsafe_yaml_str = "!!python/object/apply:os.system ['whoami']"

In [12]:
yaml = ruyaml.YAML(typ="unsafe")
yaml

In [13]:
try:
    # 加载时会执行 os.system('whoami')
    yaml.load(unsafe_yaml_str)
except Exception as e:
    print(e)

## 4. `typ="base"` (Base - 基础模式)

这是一个非常底层的模式，对应 `PyYAML` 的 `BaseLoader`。普通用户很少会用到。

- 核心目的：提供最基础的 `YAML` 解析能力，不进行大部分自动类型转换。

- 主要特性：

    - 仅加载 `YAML` 的基本结构（映射、序列和字符串）。

    - 不会自动将 `true`、`123` 等解析为布尔值或整数，它们都会被当作字符串处理，除非有显式的标签（如 `!!int 123`）。

- 适用场景：

    - 进行非常底层的 `YAML` 处理。

    - 希望完全手动控制所有类型转换。

In [14]:
yaml_str = """
# 服务器配置
server:
  host: 127.0.0.1  # 本机地址
  port: 8080
"""

In [15]:
yaml = ruyaml.YAML(typ="base")  # 明确指定安全模式
yaml

In [16]:
data = yaml.load(yaml_str)

print(type(data))  # <class 'dict'>
print(data)

<class 'dict'>
{'server': {'host': '127.0.0.1', 'port': '8080'}}


# 读写文件

In [17]:
data = {
    "server": {"host": "localhost", "port": 8080},
    "database": {"name": "testdb", "user": "root", "password": "password"},
}

In [18]:
yaml = ruyaml.YAML(typ="rt")  # 'rt' 是默认值，可以省略
yaml

In [19]:
file_path = "ruamel.yaml--example.yaml"

## dump / load

In [20]:
with open(file_path, "w", encoding="utf-8") as f:
    yaml.dump(data, f)

In [21]:
with open(file_path, "r", encoding="utf-8") as f:
    data = yaml.load(f)
data

{'server': {'host': 'localhost', 'port': 8080}, 'database': {'name': 'testdb', 'user': 'root', 'password': 'password'}}

## dump_all / load_all: 默认的 dump 和 load 是把传入的对象放入一个列表中，使用 __all 需要直接传入一个列表

In [22]:
with open(file_path, mode="w", encoding="utf-8") as f:
    yaml.dump_all([data], f)

In [23]:
data = []
with open(file_path, mode="r", encoding="utf-8") as f:
    _data = yaml.load_all(f)
    for i in _data:
        data.append(i)
data

[{'server': {'host': 'localhost', 'port': 8080}, 'database': {'name': 'testdb', 'user': 'root', 'password': 'password'}}]

# ruamel.yaml 可以在列表左侧缩进

In [24]:
data = {
    "name": ["小明", "小红"],
    "map": {"ak": "agargta", "sk": "agtyahmy"},
    "xy": [
        [88.00862548610722, 93.6939303578312],
        [88.00778259970214, 93.69252192568183],
        [88.00837307563499, 93.69132246303569],
        [88.00682235480895, 93.69322364611894],
        [88.0066984176408, 93.6932256070429],
    ],
    "xy_min_max": [0.014877027627733241, 0.011471283462284987],
}

In [25]:
yaml = ruyaml.YAML()
yaml

In [26]:
# 避免自动排序
yaml.sort_base_mapping_type_on_output = False

In [37]:
# mapping:  这个参数指定了字典（映射类型）的缩进级别。在 YAML 中，字典通常表示为键值对，这个参数控制了字典中每个键值对的缩进。
# sequence: 这个参数指定了列表（序列类型）的缩进级别。在 YAML 中，列表通常表示为一系列条目，这个参数控制了列表中每个条目的缩进。
# offset:   这个参数指定了缩进的起始偏移量。在某些情况下，你可能希望在缩进之前有一个额外的空格或其他字符，offset 参数允许你设置这个偏移量。
#           例如，如果你设置 offset=2，那么每个缩进级别将增加两个字符的宽度，这可能是为了与代码中的其他部分对齐或出于其他格式化目的。
# 此外，ruamel.yaml 还提供了其他参数来控制缩进和格式化：
# pre:       这个参数允许你为每个缩进级别指定一个前缀字符串。例如，你可以设置 pre='  ' 来使用两个空格作为每个缩进级别的前缀。
# post:      类似于 pre，这个参数允许你为每个缩进级别指定一个后缀字符串。
# transform: 这个参数允许你指定一个函数，该函数将在每个缩进级别之前应用，可以用来动态地改变缩进的字符。
yaml.indent(mapping=2, sequence=4, offset=2)

In [38]:
# 允许使用 Unicode 字符
yaml.allow_unicode = True

In [39]:
yaml.dump(data, sys.stdout)

name:
  - 小明
  - 小红
map:
  ak: agargta
  sk: agtyahmy
xy:
  -   - 88.00862548610722
      - 93.6939303578312
  -   - 88.00778259970214
      - 93.69252192568183
  -   - 88.00837307563499
      - 93.69132246303569
  -   - 88.00682235480895
      - 93.69322364611894
  -   - 88.0066984176408
      - 93.6932256070429
xy_min_max:
  - 0.014877027627733241
  - 0.011471283462284987
